# Working with Simulation Output
In this tutorial, we will take a closer look at what is in our simulation output. We will build upon our previous tutorial on using uproot and build the first steps of something we could use to analyse our data. Before we begin though, we will take a look at plotting our data as histograms, so that we can better understand what is going on. Of course, we now also have the benefit of a but of background information on what is actually going on in our detector and simulation. So now, we can begin to comprehend and interpret what our plots will show.

# Setup
You need the following packages installed: EDM4hep, Plots, LinearAlgebra, Combinatorics.

This is done simply by activating and instantiating the environment where this notebook is located.

In [ ]:
#import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
#using EDM4hep
#EDM4hep.set_edmodel("eic")  # set the EDM model to use

### Load the necessary packages

In [ ]:
using EDM4hep
using EDM4hep.RootIO
using EDM4hep.Histograms
using EDM4hep.Analysis
using Plots
using LinearAlgebra

fname = "pythia8NCDIS_18x275_minQ2=10_beamEffects_xAngle=-0.025_hiDiv_1.0000.eicrecon.edm4eic.root"
#eic_server = "root://dtn-eic.jlab.org/"
#fpath = "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC/18x275/minQ2=10/"
#fname = eic_server * fpath * "pythia8NCDIS_18x275_minQ2=10_beamEffects_xAngle=-0.025_hiDiv_1.0005.eicrecon.edm4eic.root"
reader = RootIO.Reader(fname)
events = RootIO.get(reader, "events");

## Basic Plotting Introduction
We will begin by taking a look at some of our reconstructed charged particles. The collection is called `ReconstructedChargedParticles` and the sub-branch `energy`. Note that in ROOT (and Python) you would access it with `ReconstructedChargedParticles.energy`, with UnROOT the separator is with `_`. 

Note also that the returned type for `events.ReconstructedChargedParticles_energy` is a `Vector` of `Vector`s, therefore we need to flatten them with `vcat`

In [ ]:
histogram(vcat(events.ReconstructedChargedParticles_energy...), bins=range(1,50,100))

We can separate the positively and negatively charged particles

In [ ]:
positive = vcat(events.ReconstructedChargedParticles_charge...) .> 0
negative = vcat(events.ReconstructedChargedParticles_charge...) .< 0
histogram(vcat(events.ReconstructedChargedParticles_energy...)[positive], bins=range(0,50,100))

Plot both histograms are the same time 

In [ ]:
histogram(vcat(events.ReconstructedChargedParticles_energy...)[negative], label="negative", bins=range(0,50,100), xlabel="GeV")
histogram!(vcat(events.ReconstructedChargedParticles_energy...)[positive], label="positive", bins=range(0,50,100), xlabel="GeV")

## Basic Analysis
In this section, we will make some basic analysis plots checking the detection efficiency and resolution for some of our particles. Before that though, we will take a look at associations and how we can utilise them.

In [ ]:
get_charged = RootIO.create_getter(reader, "ReconstructedChargedParticles"; selection=[:energy, :momentum, :charge])
get_asso    = RootIO.create_getter(reader, "ReconstructedChargedParticleAssociations")
get_mcps    = RootIO.create_getter(reader, "MCParticles"; selection=[:PDG, :momentum, :charge, :mass, :parents, :daughters])

# Basic resolution histogram
hresolu = H1D("Resolution [%]", 100, -10., 10., unit=:GeV)

# Loop over events and fill histogram
for evt in events
    recps  = get_charged(evt)     # Get the coll. of reconstructed charged particles
    assocs = get_asso(evt)        # Get the coll. of associations `rec <-> mcp`
    mcps   = get_mcps(evt)        # Get the coll. of MC particles. Used to get .energy

    for recp in recps             # Loop over reconstructed particles
        ind = findfirst(x -> x.rec == recp, assocs)  # Find the association to ReconstructedParticle
        isnothing(ind) && continue                   # If no association found, skip to next recp
        ΔE = recp.energy - assocs[ind].sim.energy    # Calculate energy resolution
        push!(hresolu, 100*ΔE/recp.energy)           # Fill histogram
    end
end

In [ ]:
plot(hresolu.hist, title=hresolu.title, cgrad=:plasma)

## Event model access and navigation
Object attributes can be accessed naturally with the (.) notation including relationships (1-to-1 or 1-N). See the following example with the decay products of a MCParticle.  
 

In [ ]:
mcps = get_mcps(events[1])                   # Get the collection of MC particles in the first event
for mcp in mcps                              # Loop over MC particles
    length(mcp.daughters) == 2 || continue   # only consider particles with 2 daughters
    println(mcp.name, "  Energy: ", mcp.energy, " Status: ", mcp.generatorStatus)
    for d in mcp.daughters
        println("   --->: ", d.name, "  Energy: ", d.energy)
    end
end